# Recommendation System using Collaborative Filtering

Building a basic recommendation system using user-item matrix and cosine similarity (Collaborative Filtering technique).

In [ ]:

# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:

# Step 2: Create a sample user-item rating dataset
data = {
    'User': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'D'],
    'Item': ['Item1', 'Item2', 'Item3', 'Item1', 'Item2', 'Item2', 'Item3', 'Item1'],
    'Rating': [5, 3, 4, 5, 2, 4, 5, 3]
}

df = pd.DataFrame(data)
print(df)


In [ ]:

# Step 3: Create a User-Item Matrix
user_item_matrix = df.pivot_table(index='User', columns='Item', values='Rating').fillna(0)
print("\nUser-Item Matrix:")
print(user_item_matrix)


In [ ]:

# Step 4: Calculate Cosine Similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("\nUser Similarity Matrix:")
print(user_similarity_df)


In [ ]:

# Step 5: Generate Recommendations for a target user (Example: User 'D')
def recommend_items(user, user_item_matrix, similarity_df, top_n=2):
    similar_users = similarity_df[user].sort_values(ascending=False)[1:]  # Exclude self
    
    scores = {}
    for similar_user, sim_score in similar_users.items():
        user_ratings = user_item_matrix.loc[similar_user]
        for item in user_item_matrix.columns:
            if user_item_matrix.loc[user, item] == 0 and user_ratings[item] > 0:
                scores[item] = scores.get(item, 0) + sim_score * user_ratings[item]
    
    recommended_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return [item for item, score in recommended_items]

target_user = 'D'
recommendations = recommend_items(target_user, user_item_matrix, user_similarity_df, top_n=2)
print(f"Recommended items for User {target_user}: {recommendations}")


In [ ]:

# Step 6: Evaluation (using simple metrics - Coverage and Hit Rate for demonstration)
# In real projects, larger datasets and metrics like RMSE, MAE, Precision@K, etc. are used.

def hit_rate(recommendations, actual_items):
    hits = len(set(recommendations) & set(actual_items))
    return hits / len(actual_items) if actual_items else 0

# Example test for user 'D' assuming actual liked item was 'Item2'
actual_liked_items = ['Item2']
hit = hit_rate(recommendations, actual_liked_items)
print(f"Hit Rate for User {target_user}: {hit:.2f}")
